In [1]:
from diffUV import dyned_eul, dyn_body, kin, dyned_quat
from diffUV.utils.symbols import *
from casadi import *
from blue_rov import Params as ps
import numpy as np
import os
from scipy.spatial.transform import Rotation as R

In [2]:
r_g = vertcat(x_g, y_g, z_g) # center of gravity wrt body origin
r_b = vertcat(x_b, y_b, z_b) # center of buoyancy wrt body origin 
I_o = vertcat(I_x, I_y, I_z,I_xz) # rigid body inertia wrt body origin

decoupled_added_m = vertcat(X_du, Y_dv, Z_dw, K_dp, M_dq, N_dr) # added mass in diagonals
coupled_added_m =  vertcat(X_dq, Y_dp, N_dp, M_du, K_dv) # effective added mass in non diagonals

linear_dc = vertcat(X_u, Y_v, Z_w, K_p,  M_q, N_r) # linear damping coefficients
quadratic_dc = vertcat(X_uu, Y_vv, Z_ww, K_pp, M_qq, N_rr) # quadratic damping coefficients

n0 = vertcat(n, dn) # state variables wrt ned
v0 = x_nb # velocity state variables wrt body

<h1>Kinematics ned Transformation Matrix</h1>

In [3]:
Kinematics = kin()
Jk = Kinematics.J
J_inv = Kinematics.J_inv
print(J_inv)
J_inv_func = Function('J_inv', [eul], [J_inv]) # for numerical & symbolic use

@1=0, 
[[(cos(psi)*cos(phi)), (sin(psi)*cos(phi)), (-sin(phi)), @1, @1, @1], 
 [(((cos(psi)*sin(phi))*sin(thet))-(sin(psi)*cos(thet))), ((cos(psi)*cos(thet))+((sin(thet)*sin(phi))*sin(psi))), (cos(phi)*sin(thet)), @1, @1, @1], 
 [((sin(psi)*sin(thet))+((cos(psi)*cos(thet))*sin(phi))), (((sin(phi)*sin(psi))*cos(thet))-(cos(psi)*sin(thet))), (cos(phi)*cos(thet)), @1, @1, @1], 
 [@1, @1, @1, 1, 00, (-sin(phi))], 
 [@1, @1, @1, 00, cos(thet), (cos(phi)*sin(thet))], 
 [@1, @1, @1, 00, (-sin(thet)), (cos(phi)*cos(thet))]]


In [4]:
# example usage of J mat
jinv = np.array(J_inv_func([0.13,0.17,0.1]))
jinv

array([[ 0.98066095,  0.09839429, -0.16918235,  0.        ,  0.        ,
         0.        ],
       [-0.07716877,  0.98879774,  0.12776544,  0.        ,  0.        ,
         0.        ],
       [ 0.17985851, -0.11223898,  0.9772683 ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        -0.16918235],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.99156189,
         0.12776544],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.12963414,
         0.9772683 ]])

In [5]:
# body representaion
uv_body = dyn_body()

# ned representaion
uv_ned = dyned_eul()

# quaternion representation
uv_quat = dyned_quat()

In [6]:
x_nb

SX([u, v, w, p, q, r])

In [7]:

def linear_vel_R(eul):
    phi = eul[2]
    thet = eul[1]
    psi = eul[0]
    R = np.zeros((3,3))
    R[0,0] = np.cos(psi)*np.cos(thet)
    R[0,1] = -np.sin(psi)*np.cos(phi) + np.cos(psi)*np.sin(thet)*np.sin(phi)
    R[0,2] = np.sin(psi)*np.sin(phi) + np.cos(psi)*np.cos(phi)*np.sin(thet)

    R[1,0] = np.sin(psi)*np.cos(thet)
    R[1,1] = np.cos(psi)*np.cos(phi) + np.sin(phi)*np.sin(thet)*np.sin(psi)
    R[1,2] = -np.cos(psi)*np.sin(phi) + np.sin(thet)*np.sin(psi)*np.cos(phi)

    R[2,0] = -np.sin(thet)
    R[2,1] = np.cos(thet)*np.sin(phi)
    R[2,2] = np.cos(thet)*np.cos(phi)
    return R

In [8]:
def linear_vel_Rq(uq):
    eps1 = uq[0]
    eps2 = uq[1]
    eps3 = uq[2]
    eta = uq[3]

    Rq = np.zeros((3,3))
    Rq[0,0] = 1 - 2*(eps2**2 + eps3**2)
    Rq[0,1] = 2*(eps1*eps2 - eps3*eta)
    Rq[0,2] = 2*(eps1*eps3 + eps2*eta)
    Rq[1,0] = 2*(eps1*eps2 + eps3*eta)
    Rq[1,1] = 1 - 2*(eps1**2 + eps3**2)
    Rq[1,2] = 2*(eps2*eps3 - eps1*eta)
    Rq[2,0] = 2*(eps1*eps3 - eps2*eta)
    Rq[2,1] = 2*(eps2*eps3 + eps1*eta)
    Rq[2,2] = 1 - 2*(eps1**2 + eps2**2)
    return Rq

In [16]:
# state transformations
J_ = uv_ned.J
Jq_ = uv_quat.J

J_func = Function('J_', [eul], [J_[:3,:3]])
Jq_func = Function('Jq_', [uq], [Jq_[:3,:3]])
Jk_func = Function('J', [eul], [Jk[:3,:3]]) # for numerical & symbolic use

# x_nb_v = np.array([0.5, 0.5, 0.5, 0, 0, 0])
r = R.from_euler("ZYX", (15, 0, 0), degrees=True)
eul_v = r.as_euler("ZYX")
uq_v = r.as_quat()

print(uq_v)
# eul_v
# uq_v

# linear_vel_R(eul_v) , r.as_matrix(), linear_vel_Rq(uq_v)

J_func(eul_v), Jq_func(uq_v) , Jk_func(eul_v) , r.as_matrix(), linear_vel_R(eul_v)

[0.         0.         0.13052619 0.99144486]


(DM(
 [[1, 0, 0], 
  [0, 0.965926, -0.258819], 
  [-0, 0.258819, 0.965926]]),
 DM(
 [[-1, 0, 0], 
  [0, -0.965926, 0.258819], 
  [0, 0.258819, 0.965926]]),
 DM(
 [[1, 0, 0], 
  [0, 0.965926, -0.258819], 
  [-0, 0.258819, 0.965926]]),
 array([[ 0.96592583, -0.25881905,  0.        ],
        [ 0.25881905,  0.96592583,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ 0.96592583, -0.25881905,  0.        ],
        [ 0.25881905,  0.96592583,  0.        ],
        [-0.        ,  0.        ,  1.        ]]))

<h1>System Inertia Matrix example</h1>

In [7]:
inertia_mat = uv_body.body_inertia_matrix()
M_func = Function('M_b', [m, I_o, z_g, decoupled_added_m, coupled_added_m], [inertia_mat]) # for numerical & symbolic use

In [8]:
# c , cpp or matlab code generation
M_func = Function('M_b', [m, I_o, z_g, decoupled_added_m, coupled_added_m], [inertia_mat]) # for both numerical & symbolic use
M_func.generate("M_b.c")
os.system(f"gcc -fPIC -shared M_b.c -o libM_b.so")

0

In [9]:
# example usage of inertia mat
M_mat = np.array(M_func(ps.m, ps.Io, ps.rg[2], ps.added_m, ps.coupl_added_m))
M_mat

array([[17.  ,  0.  ,  0.  ,  0.  ,  0.23,  0.  ],
       [ 0.  , 24.2 ,  0.  , -0.23,  0.  ,  0.  ],
       [ 0.  ,  0.  , 26.07,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.23,  0.  ,  0.28,  0.  ,  0.  ],
       [ 0.23,  0.  ,  0.  ,  0.  ,  0.28,  0.  ],
       [ 0.  ,  0.  ,  0.  , -0.  ,  0.  ,  0.28]])

In [10]:
inertia_ned_mat = uv_ned.ned_euler_inertia_matrix()
Onb = vertcat(thet, phi, psi)
M_ned_func = Function('M_ned', [m, I_o, z_g, Onb, decoupled_added_m, coupled_added_m], [inertia_ned_mat]) # for numerical & symbolic use

In [11]:
# example usage of inertia mat in ned
Onb_v = np.array([0.13,0.17,0.1])
np.array(M_ned_func(ps.m, ps.Io, ps.rg[2], Onb_v,  ps.added_m, ps.coupl_added_m))

array([[ 1.83362708e+01, -7.03041061e+00,  3.71555407e+00,
         1.77488172e-02,  2.23648786e-01,  2.58149654e-02],
       [-7.03041061e+00,  4.90893516e+01, -2.23061182e+01,
        -2.27423481e-01,  2.24397276e-02,  4.13674584e-02],
       [ 3.71555407e+00, -2.23061182e+01,  2.57798665e+01,
        -2.93860503e-02, -3.85835972e-02, -8.67361738e-19],
       [ 1.77488172e-02, -2.27423481e-01, -2.93860503e-02,
         2.80000000e-01,  0.00000000e+00, -4.73710577e-02],
       [ 2.23648786e-01,  2.24397276e-02, -3.85835972e-02,
         0.00000000e+00,  2.80000000e-01,  0.00000000e+00],
       [ 2.58149654e-02,  4.13674584e-02,  8.67361738e-19,
        -4.73710577e-02,  0.00000000e+00,  2.80000000e-01]])

In [12]:
# checks --> above results (inertia mat in ned using body inertia and J tansform)
jinv.T@M_mat@jinv 

array([[ 1.83362708e+01, -7.03041061e+00,  3.71555407e+00,
         1.77488172e-02,  2.23648786e-01,  2.58149654e-02],
       [-7.03041061e+00,  4.90893516e+01, -2.23061182e+01,
        -2.27423481e-01,  2.24397276e-02,  4.13674584e-02],
       [ 3.71555407e+00, -2.23061182e+01,  2.57798665e+01,
        -2.93860503e-02, -3.85835972e-02, -7.71263355e-19],
       [ 1.77488172e-02, -2.27423481e-01, -2.93860503e-02,
         2.80000000e-01,  0.00000000e+00, -4.73710577e-02],
       [ 2.23648786e-01,  2.24397276e-02, -3.85835972e-02,
         0.00000000e+00,  2.80000000e-01, -2.51507399e-18],
       [ 2.58149654e-02,  4.13674584e-02,  7.71263355e-19,
        -4.73710577e-02, -2.45961687e-18,  2.80000000e-01]])

<h1>Forward dynamics simulation example</h1>

In [13]:
#euler
forward_dynamics = uv_ned.ned_euler_forward_dynamics()
print(forward_dynamics)
print(forward_dynamics.size())
fd_func = Function('f', [m, W, B, r_g, r_b, I_o, 
                   decoupled_added_m, coupled_added_m, 
                   linear_dc, quadratic_dc,
                   v0, n0, tau_b], [forward_dynamics])

@1=(cos(psi)*cos(phi)), @2=(m-X_du), @3=(@1*@2), @4=(((cos(psi)*sin(phi))*sin(thet))-(sin(psi)*cos(thet))), @5=(m-Y_dv), @6=(@4*@5), @7=((sin(psi)*cos(thet))+((cos(psi)*cos(thet))*sin(phi))), @8=(m-Z_dw), @9=(@7*@8), @10=(sin(psi)*cos(phi)), @11=(@10*@2), @12=((cos(psi)*cos(thet))+((sin(thet)*sin(phi))*sin(psi))), @13=(@12*@5), @14=(((sin(phi)*sin(psi))*sin(thet))-(cos(psi)*cos(thet))), @15=(@14*@8), @16=(cos(phi)*sin(thet)), @17=(@16*@5), @18=sin(phi), @19=(@18*@2), @20=(cos(phi)*cos(thet)), @21=(@20*@8), @22=((m*z_g)+K_dv), @23=cos(thet), @24=((m*z_g)-M_du), @25=(@23*@24), @26=(cos(phi)*sin(thet)), @27=(@26*@24), @28=sin(phi), @29=(@28*@22), @30=((m*z_g)+Y_dp), @31=(@4*@30), @32=(@12*@30), @33=(@16*@30), @34=(I_x-K_dp), @35=sin(thet), @36=(I_xz+N_dp), @37=(@35*@36), @38=(cos(phi)*cos(thet)), @39=((@28*@34)+(@38*@36)), @40=((m*z_g)-X_dq), @41=(@1*@40), @42=(@10*@40), @43=(@18*@40), @44=(I_y-M_dq), @45=(@23*@44), @46=(I_z-N_dr), @47=(@35*@46), @48=(@26*@44), @49=(@38*@46), 
[[(((@3*@1)

In [14]:
# quaternion
forward_dynamics_q = uv_quat.forward_dynamics_ned_quat()
print(forward_dynamics_q)
print(forward_dynamics_q.size())
fdq_func = Function('fq', [m, W, B, r_g, r_b, I_o,
                           decoupled_added_m, coupled_added_m,
                           linear_dc, quadratic_dc,
                           v0, n, uq, tau_b], [forward_dynamics_q])

@1=1, @2=2, @3=(@1-(@2*(sq(eps2)+sq(eps3)))), @4=(m-X_du), @5=(@3*@4), @6=(@2*((eps1*eps2)-(eps3*eta))), @7=(m-Y_dv), @8=(@6*@7), @9=(@2*((eps1*eps3)+(eps2*eta))), @10=(m-Z_dw), @11=(@9*@10), @12=(@2*((eps1*eps2)+(eps3*eta))), @13=(@12*@4), @14=(@1-(@2*(sq(eps1)+sq(eps3)))), @15=(@14*@7), @16=(@2*((eps2*eps3)-(eps1*eta))), @17=(@16*@10), @18=(@2*((eps1*eps3)-(eps2*eta))), @19=(@18*@4), @20=(@2*((eps2*eps3)+(eps1*eta))), @21=(@20*@7), @22=(@1-(@2*(sq(eps1)+sq(eps2)))), @23=(@22*@10), @24=4, @25=0.5, @26=(@24*(@25*eps1)), @27=((m*z_g)+K_dv), @28=(@26*@27), @29=(@24*(@25*eps2)), @30=((m*z_g)-M_du), @31=(@29*@30), @32=(@24*(@25*eps3)), @33=(@32*@30), @34=(@24*(@25*eta)), @35=(@34*@27), @36=(@24*(@25*eta)), @37=(@36*@30), @38=(@24*(@25*eps3)), @39=(@38*@27), @40=(@24*(@25*eps1)), @41=(@40*@30), @42=(@24*(@25*eps2)), @43=(@42*@27), @44=((m*z_g)+Y_dp), @45=(@6*@44), @46=((m*z_g)-X_dq), @47=(@3*@46), @48=(@14*@44), @49=(@12*@46), @50=(@20*@44), @51=(@18*@46), @52=(I_y-M_dq), @53=(@29*@52), @54

In [13]:
# example usage of forward dynamics in quaternion
tau = np.array([0.0,0,0,0,0,0])
v0_v = np.array([0, 0, 0 ,0 ,0, 0])
n_v = np.array([0, 0, 0, 0, 0, 0])
uq_v = np.array([0, 0, 0, 1])
fdq_func(ps.m, ps.W, ps.B, ps.rg, ps.rb, ps.Io, ps.added_m, ps.coupl_added_m, ps.linear_dc,ps.quadratic_dc, v0_v,  n_v, uq_v, tau)

DM(
[[17, 0, 0, 0, 0.46, -0, 0], 
 [0, 24.2, 0, -0, -0, 0.46, 0], 
 [0, 0, 26.07, 0, -0, 0, 0], 
 [0, -0, 0, 1.12, 0, 0, -0], 
 [0.46, -0, -0, 0, 1.12, 0, 0], 
 [-0, 0.46, 0, 0, 0, 1.12, 0], 
 [0, 0, 0, 0, 0, 0, 0]])

In [14]:
# example usage of forward dynamics
tau = np.array([0.0,0,0,0,0,0])
v0_v = np.array([0, 0, 0 ,0 ,0, 0])
n0_v = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
fd_func(ps.m, ps.W, ps.B, ps.rg, ps.rb, ps.Io, ps.added_m, ps.coupl_added_m, ps.linear_dc,ps.quadratic_dc, v0_v,  n0_v,  tau)

DM(
[[17, 0, 0, -0, 0.23, 0], 
 [0, 50.27, -26.07, -0.23, 0, 0], 
 [0, -26.07, 26.07, -0, -0, 0], 
 [-0, -0.23, -0, 0.28, 0, -0], 
 [0.23, 0, -0, 0, 0.28, 0], 
 [0, 0, 0, -0, 0, 0.28]])

In [14]:
# forward dynamics for integrator
v0_n = J_inv@dn

ode_q_dot = dn
ode_q_ddot = fd_func(ps.m, ps.W, ps.B, ps.rg, ps.rb, ps.Io, ps.added_m, ps.coupl_added_m, ps.linear_dc,ps.quadratic_dc, v0_n, n0, tau_b)
ode = vertcat(ode_q_dot, ode_q_ddot) #the complete ODE vector
f_ode = Function('Odefunc', [n0, tau_b], [ode])

In [15]:
T = 10 # time horizon
N = 20 # number of control intervals

# integrator to discretize the system
dae = {'x':n0, 'p':tau_b, 'ode':f_ode(n0,tau_b)}
intg = integrator('intg', 'rk', dae, {'tf':T/N, 'number_of_finite_elements':4, 'simplify':True})

CasADi - 2024-06-13 18:12:28 WARNING("The options 't0', 'tf', 'grid' and 'output_t0' have been deprecated.
The same functionality is provided by providing additional input arguments to the 'integrator' function, in particular:
 * Call integrator(..., t0, tf, options) for a single output time, or
 * Call integrator(..., t0, grid, options) for multiple grid points.
The legacy 'output_t0' option can be emulated by including or excluding 't0' in 'grid'.
Backwards compatibility is provided in this release only.") [.../casadi/core/integrator.cpp:521]


In [16]:
res = intg(x0=n0,p=tau_b) #evaluate with symbols
x_next = res['xf'] 

In [17]:
# Simplify API to (x,u)->(x_next)
F = Function('F',[n0,tau_b],[x_next])

In [27]:
tau = np.array([0.0001, 0.0, 0.0, 0.0, 0.0, 0.0])
# n0_v = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
n0_v = F(n0_v, tau)
n0_v[3:6]

DM([0.202826, -0.15293, -0.000726845])